In [13]:
import censusgeocode as cg
import pandas as pd

from dialysis.constants import DATA_DIR

In [4]:
df = pd.read_csv(DATA_DIR / "dialysis_current_facilities" / "ICH_CAHPS_FACILITY.csv")

In [10]:
# It turns out there's a very few number of chains
df["Chain Organization"].value_counts()

DaVita                                         2970
Fresenius Medical Care                         2801
Independent                                     733
US Renal Care Inc.                              331
Dialysis Clinic Inc.                            262
American Renal Associates                       238
Other                                           148
Satellite Healthcare                             58
Diversified Specialty Institutes (DSI)           23
Northwest Kidney Centers                         20
State Owned                                      19
Kaiser Permanente                                19
Centers for Dialysis Care                        19
Wake Forest University                           18
Atlantis Healthcare Group                        17
Satellite Dialysis                               16
Greenfield Health Systems                        15
University of Utah Dialysis Program              14
Sanford Health                                   13
Mayo Clinic 

In [21]:
len(df)

7830

In [25]:
# Geocode all the results. Takes about 30min

data = [
    {
        "id": i,
        "street": row["Address Line 1"],
        "city": row["City"],
        "state": row["State"],
        "zip": row["Zip"],
    }
    for i, row in df.iterrows()
]

resp = cg.addressbatch(data)

In [34]:
# Merge the geocoded data back
df_resp = pd.DataFrame(resp)
df_resp["id"] = df_resp["id"].astype(int)

merged_df = df.merge(
    df_resp,
    left_index=True,
    right_on="id",
    how="left",
    indicator=True,
)

In [42]:
# Make sure we didn't miss anyting in the merge
merged_df["_merge"].value_counts()

both          7830
left_only        0
right_only       0
Name: _merge, dtype: int64

In [44]:
# Write out the data

merged_df.to_parquet(DATA_DIR / "dialysis_facilities_geocoded.parquet")